In [25]:
import pandas as pd
import numpy as np
import os

def filepath(f):
    d = os.path.join(os.getcwd(), 'data', 'fraud_detection_data', f)
    return d    

# Importing testing datasets
test_inpatient = pd.read_csv(filepath("Test_Inpatientdata.csv"))
test_outpatient = pd.read_csv(filepath("Test_Outpatientdata.csv"))
test_beneficiary = pd.read_csv(filepath("Test_Beneficiarydata.csv"))
test_label = pd.read_csv(filepath("Test.csv"))

In [27]:
##Replacing 2 with 0 for chronic conditions ,that means chroniv condition No is 0 and yes is 1
test_beneficiary = test_beneficiary.replace({'ChronicCond_Alzheimer': 2, 'ChronicCond_Heartfailure': 2, 'ChronicCond_KidneyDisease': 2,
                           'ChronicCond_Cancer': 2, 'ChronicCond_ObstrPulmonary': 2, 'ChronicCond_Depression': 2, 
                           'ChronicCond_Diabetes': 2, 'ChronicCond_IschemicHeart': 2, 'ChronicCond_Osteoporasis': 2, 
                           'ChronicCond_rheumatoidarthritis': 2, 'ChronicCond_stroke': 2 }, 0)

test_beneficiary = test_beneficiary.replace({'RenalDiseaseIndicator': 'Y'}, 1)

In [28]:
#Combining test data
test_inpatient['inpatient'] = 1
test_outpatient['inpatient'] = 0
#test = pd.concat([test_inpatient, test_outpatient], axis=0, sort=False)
test=pd.merge(test_outpatient,test_inpatient,
                              left_on=['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode','inpatient'],
                              right_on=['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2',
       'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5',
       'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8',
       'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10', 'ClmProcedureCode_1',
       'ClmProcedureCode_2', 'ClmProcedureCode_3', 'ClmProcedureCode_4',
       'ClmProcedureCode_5', 'ClmProcedureCode_6', 'DeductibleAmtPaid',
       'ClmAdmitDiagnosisCode','inpatient']
                              ,how='outer')

test_alldata=pd.merge(test,test_beneficiary,left_on='BeneID',right_on='BeneID',how='inner')


C:\Users\aengu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\reshape\merge.py:1213: RuntimeWarning: invalid value encountered in cast
  if not (rk == rk.astype(lk.dtype))[~np.isnan(rk)].all():


In [29]:
# Drop constant columns
for col in test_alldata.columns:
    if test_alldata[col].nunique() <= 1:
        test_alldata.drop([col], inplace=True, axis=1)
        print(col, "dropped as it is a constant.")

ClmProcedureCode_6 dropped as it is a constant.


In [30]:
#Merging with Provider
test_allprovider=pd.merge(test_label,test_alldata,on='Provider')

In [ ]:
def check_alive(x):
    if pd.isnull(x): # 
        return 1
    else:
        return 0

test_allprovider["is_alive"] = test_allprovider["DOD"].apply(check_alive)

In [31]:
# Process datetime columns -- Get age of claimant
test_allprovider["DOD"] = pd.to_datetime(test_allprovider["DOD"])
test_allprovider["DOB"] = pd.to_datetime(test_allprovider["DOB"])

latest_date = pd.to_datetime(test_allprovider["ClaimEndDt"]).max()
latest_date

Timestamp('2009-12-31 00:00:00')

In [32]:
# Filling the NA values in DOD column of alive claimants with latest claim date
test_allprovider["DOD"] = test_allprovider["DOD"].fillna(latest_date)
test_allprovider["age"] = (test_allprovider["DOD"] - test_allprovider["DOB"])/np.timedelta64(1,'Y')

In [33]:
# Get claim duration column and remove datetime columns of claim start and end
test_allprovider["claim_duration"] = (pd.to_datetime(test_allprovider["ClaimEndDt"]) - pd.to_datetime(test_allprovider["ClaimStartDt"]))/np.timedelta64(1,'D')
test_allprovider["claim_duration"] = test_allprovider['claim_duration'].astype('float64')

# Handle time spent in hospital
test_allprovider["time_under_care"] = (pd.to_datetime(test_allprovider["DischargeDt"]) - pd.to_datetime(test_allprovider["AdmissionDt"]))/np.timedelta64(1,'D')
test_allprovider["time_under_care"] = test_allprovider['time_under_care'].astype('float64')
test_allprovider["time_under_care"] = test_allprovider["time_under_care"].fillna(0.0)

In [34]:
#Removing unecessary columns
drop_cols = [
    'NoOfMonths_PartACov', 
    'NoOfMonths_PartBCov', 
    'State', 
    'County', 
    'AttendingPhysician',
    'OperatingPhysician', 
    'OtherPhysician', 
    'DiagnosisGroupCode',
    'DOB',
    'DOD',
    'ClaimStartDt',
    'ClaimEndDt',
    'BeneID',
    'AdmissionDt',
    'DischargeDt'
    ]
test_allprovider=test_allprovider.drop(drop_cols, axis=1)

In [35]:
#Read in icd9 data
diagnosis=pd.read_csv("data/medical_codes_data/icd9_diagnosis.csv", dtype={'code': str})
procedures=pd.read_csv("data/medical_codes_data/icd9_procedure.csv", dtype={'code': str})

In [36]:
diagnosis_dict = {}
unique_diagnosis = diagnosis['description_1'].unique()
for (i,row) in diagnosis.iterrows():
    diagnosis_dict[row["code"]] = row["description_1"]

procedures_dict = {}
unique_procedures = procedures['description_1'].unique()
for (i,row) in procedures.iterrows():
    procedures_dict[row["code"]] = row["description_1"]


In [37]:
unique_procedures

array(['Procedures And Interventions , Not Elsewhere Classified',
       'Operations On The Nervous System',
       'Operations On The Endocrine System', 'Operations On The Eye',
       'Other Miscellaneous Diagnostic And Therapeutic Procedures',
       'Operations On The Ear',
       'Operations On The Nose, Mouth, And Pharynx',
       'Operations On The Respiratory System',
       'Operations On The Cardiovascular System',
       'Operations On The Hemic And Lymphatic System',
       'Operations On The Digestive System',
       'Operations On The Urinary System',
       'Operations On The Male Genital Organs',
       'Operations On The Female Genital Organs',
       'Obstetrical Procedures',
       'Operations On The Musculoskeletal System',
       'Operations On The Integumentary System',
       'Miscellaneous Diagnostic And Therapeutic Procedures'],
      dtype=object)

In [38]:
diagnoses_code_in_table = {}
for i,d in enumerate(unique_diagnosis):
    diagnoses_code_in_table[d] = i+1

procedures_code_in_table = {}
for i,p in enumerate(unique_procedures):
    procedures_code_in_table[p] = i+1



In [39]:
# Create a list of columns to process
columns_to_process = ['ClmDiagnosisCode_' + str(i) for i in range(1, 11)]
not_present_set_diagnosis = set()

# Replace the codes with the descriptions
diagnosis_descriptor = []
for (i,row) in test_allprovider.iterrows():
    descriptors = set()
    for col in columns_to_process:
        adm = False
        try:
            diag_num = str(diagnosis_dict[row[col]])
            new_code2 = diagnoses_code_in_table[diag_num]
        except:
            if pd.isnull(row[col]):
                continue
            else:
                set.add(not_present_set_diagnosis, row[col])
                continue
        descriptors.add(new_code2)
    diagnosis_descriptor.append(descriptors)


In [40]:
adm_diag = []
for (i,row) in test_allprovider.iterrows():
    ind = False
    value = row["ClmAdmitDiagnosisCode"]
    for col in columns_to_process:
        if ind == True:
            continue
        if pd.isnull(value):
            continue
        if str(value) == str(row[col]):
            adm_diag.append(1)
            ind = True
    if ind == False:
        adm_diag.append(0)
len(adm_diag)

test_allprovider["admitDiagInFinalDiagnosis"] = adm_diag

In [41]:
columns_to_process2 = ['ClmProcedureCode_' + str(i) for i in range(1, 6)]

not_present_set_procedure = set()
procedures_descriptor = []
for (i,row) in test_allprovider.iterrows():
    procedures = set()
    for col in columns_to_process2:
        try:
            n = str(int(row[col]))
            new_code = procedures_dict[n]
            new_code2 = procedures_code_in_table[new_code]
        except:
            if pd.isnull(row[col]):
                continue
            else:
                set.add(not_present_set_procedure, str(int(row[col])))
                continue
        procedures.add(new_code2)
    procedures_descriptor.append(procedures)

In [42]:
not_present_set_procedure

{'309'}

In [43]:
diagnoses_columns = {}

for j in [i for i in diagnoses_code_in_table.values()]:
    diagnoses_columns[j] = []

for d in diagnosis_descriptor:
    for k,v in diagnoses_columns.items():
        if k in d:
            v.append(1)
        else:
            v.append(0)

procedures_columns = {}

for j in [i for i in procedures_code_in_table.values()]:
    procedures_columns[j] = []

for p in procedures_descriptor:
    for k,v in procedures_columns.items():
        if k in p:
            v.append(1)
        else:
            v.append(0)
    

In [44]:
final_diagnosis_cols = {}
for k, name in zip(diagnoses_columns.keys(), diagnoses_code_in_table.values()):
    new_name = "diagnosis_"+str(name)
    final_diagnosis_cols[new_name] = diagnoses_columns[k]

final_procedures_cols = {}
for k, name in zip(procedures_columns.keys(), procedures_code_in_table.values()):
    new_name = "procedure_"+str(name)
    final_procedures_cols[new_name] = procedures_columns[k]


In [45]:
test_allprovider['diagnosis_descriptor'] = diagnosis_descriptor
test_allprovider['procedures_descriptor'] = procedures_descriptor
test_allprovider.drop(
    columns_to_process + columns_to_process2,
    axis = 1,
    inplace = True
)

test_allprovider.drop(
    ["ClmAdmitDiagnosisCode"],
    axis = 1,
    inplace = True
)

test_allprovider.head()

,Provider,ClaimID,InscClaimAmtReimbursed,DeductibleAmtPaid,inpatient,Gender,Race,RenalDiseaseIndicator,ChronicCond_Alzheimer,ChronicCond_Heartfailure,...,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,age,claim_duration,time_under_care,admitDiagInFinalDiagnosis,diagnosis_descriptor,procedures_descriptor
0,PRV51002,CLM129901,100,0.0,0,2,1,0,0,0,...,0,0,1310,590,75.084362,0.0,0.0,0,"{8, 16, 19}",{}
1,PRV51002,CLM163936,40,0.0,0,2,1,0,0,1,...,22000,3204,1020,910,88.913530,0.0,0.0,0,"{16, 19, 5}",{}
2,PRV51002,CLM348696,70,0.0,0,2,1,0,0,1,...,22000,3204,1020,910,88.913530,0.0,0.0,0,{19},{}
3,PRV51002,CLM500906,100,0.0,0,2,1,0,0,1,...,22000,3204,1020,910,88.913530,0.0,0.0,0,{7},{}
4,PRV51002,CLM272820,400,0.0,0,2,1,0,1,0,...,8000,13136,1520,730,48.997584,0.0,0.0,0,{2},{}


In [46]:
final_test_df = pd.concat(
    [
        pd.concat(
            [
                test_allprovider,
                pd.DataFrame(
                    final_diagnosis_cols
                )
            ],
            axis = 1
        ),
        pd.DataFrame(
            final_procedures_cols
        )
    ], 
    axis = 1
)

In [47]:
final_test_df.head()

,Provider,ClaimID,InscClaimAmtReimbursed,DeductibleAmtPaid,inpatient,Gender,Race,RenalDiseaseIndicator,ChronicCond_Alzheimer,ChronicCond_Heartfailure,...,procedure_9,procedure_10,procedure_11,procedure_12,procedure_13,procedure_14,procedure_15,procedure_16,procedure_17,procedure_18
0,PRV51002,CLM129901,100,0.0,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,PRV51002,CLM163936,40,0.0,0,2,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,PRV51002,CLM348696,70,0.0,0,2,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,PRV51002,CLM500906,100,0.0,0,2,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,PRV51002,CLM272820,400,0.0,0,2,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
final_test_df.to_csv("processed_data/final_test_set.csv")